In [11]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

variables = {
    'k1': 1.45e-2,
    'k2': 2.76e-1,
    'k3': 6.07e-3,
    'k4': 2.35e-4,
    'k5': 9.49e-2,
    'k6': 1.93e-1,
    'k7': 1.15,
    'k8': 7.27,
    'k9': 0,
    'k10': 0,
    'k11': 3.83e-2,
    'k12': 2.84e-1,
    'k13': 0,
    'sigma': 1.34,
    'K_M': 13.2,
    'Gbpl': 5,
    'Ibpl': 5,
    'gbliv': 0.043,
    'Gthpl': 9,
    'vG': 17/70,
    'vI': 13/70,
    'beta': 1,
    'f': 0.005551,
    'tau_i': 31,
    't_int': 30,
    'tau_d': 3,
    'c1': 0.1,
    'h': 1,
    't_half': 1,
    'a': 1,
    'b': 1
}

usa = lambda t: 1

inputs = {
    'Dmeal': 75,
    'Mb': 150,
    'usa': usa,
    'Ula': 1,
    'h': 2
}

def MGgut(t):
    D_meal = inputs['Dmeal']
    k1 = variables['k1']
    sigma = variables['sigma']
    return D_meal * np.exp(-(k1*t)**sigma)

def dGpl_dt(t, Gpl, Ipl, Irem, Usc1, Usc2):
    def gliv(Gpl, Irem):
        gbliv = variables['gbliv']
        k3 = variables['k3']
        Gbpl = variables['Gbpl']
        k4 = variables['k4']
        beta = variables['beta'] 
        return gbliv - k3 * (Gpl - Gbpl) - k4 * beta * Irem

    def ggut(MGgut):
        k2 = variables['k2']
        f = variables['f']
        vG = variables['vG']
        Mb = inputs['Mb']
        return k2 * (f / (vG * Mb)) * MGgut

    def gnonit(Gpl):
        gbliv = variables['gbliv']
        KM = variables['K_M']
        Gbpl = variables['Gbpl']
        return gbliv * ((KM + Gbpl) / Gbpl) * (Gpl / (KM + Gpl))

    def git(Gpl, Irem):
        k5 = variables['k5']
        beta = variables['beta']
        KM = variables['K_M']
        return k5 * beta * Irem * (Gpl / (KM + Gpl))

    def gren(Gpl):
        c1 = variables['c1']
        vG = variables['vG']
        Mb = inputs['Mb']
        Gthpl = variables['Gthpl']
        return 0 if Gpl < Gthpl else (c1 / (vG * Mb)) * (Gpl - Gthpl)

    return gliv(Gpl, Irem) + ggut(MGgut(t)) - gnonit(Gpl) - git(Gpl, Irem) - gren(Gpl)

def dUsc1_dt(t, Usc1):
        k10 = variables['k10']
        usa = inputs['usa']
        return usa(t) - k10 * Usc1

def dUsc2_dt(t, Usc1, Usc2):
        k9 = variables['k9']
        k10 = variables['k10']
        return k10 * Usc1 - k9 * Usc2

def dIpl_dt(t, Gpl, Ipl, Irem, Usc1, Usc2):
    def ipnc(t, Gpl, dGpl_dt):
        Gbpl = variables['Gbpl']
        beta = variables['beta']
        k6 = variables['k6']
        k7 = variables['k7']
        tau_i = variables['tau_i']
        k8 = variables['k8']
        tau_d = variables['tau_d']

        if np.isscalar(dGpl_dt):
            dGpl_dt = np.array([dGpl_dt])  # Convert scalar to array

        integral_term = np.trapz(np.squeeze(Gpl - Gbpl), t)  # Remove unnecessary dimensions
        return (1 / beta) * (k6 * np.squeeze(Gpl - Gbpl) + (k7 / tau_i) * integral_term + (k7 / tau_i) * Gbpl + (k8 * tau_d) * dGpl_dt)



    def isa(t, Usc2):
        k9 = variables['k9']
        vI = variables['vI']
        Mb = inputs['Mb']
        return k9 * (1 / (vI * Mb)) * Usc2

    def ila(t, Ula):
        h = variables['h']
        t_half = variables['t_half']
        vI = variables['vI']
        Mb = inputs['Mb']
        return ((h * (t_half ** h) * (t ** (h - 1))) / (((t_half ** h) + (t ** h)) ** 2)) * ((1 / (vI * Mb)) * Ula(t))

    def iliv(t, Gpl, Ipl):
        k7 = variables['k7']
        beta = variables['beta']
        tau_i = variables['tau_i']
        Ibpl = variables['Ibpl']
        return k7 * (variables['Gbpl'] / (beta * tau_i * Ibpl)) * Ipl

    def irem(t, Ipl):
        k11 = variables['k11']
        Ibpl = variables['Ibpl']
        return k11 * (Ipl - Ibpl)

    return ipnc(t, Gpl, dGpl_dt(t, Gpl, Ipl, Irem, Usc1, Usc2)) + isa(t, Usc2) + ila(t, inputs['Ula']) - iliv(t, Gpl, Ipl) - irem(t, Ipl)



def dIrem_dt(t, Gpl, Ipl, Irem, Usc1, Usc2):
    def ipl(t, Ipl):
        k11 = variables['k11']
        Ibpl = variables['Ibpl']
        return k11 * (Ipl - Ibpl)

    def iit(t, Irem):
        k12 = variables['k12']
        return k12 * Irem

    return ipl(t, Ipl) - iit(t, Irem)

                                                                              
def system_of_equations(t, y):
    Gpl, Ipl, Irem, Usc1, Usc2 = y

    dGpl_dt_val = dGpl_dt(t, Gpl, Ipl, Irem, Usc1, Usc2)
    dUsc1_dt_val = dUsc1_dt(t, Usc1)
    dUsc2_dt_val = dUsc2_dt(t, Usc1, Usc2)
    dIpl_dt_val = dIpl_dt(t, Gpl, Ipl, Irem, Usc1, Usc2)
    dIrem_dt_val = dIrem_dt(t, Gpl, Ipl, Irem, Usc1, Usc2)

    return [dGpl_dt_val, dIpl_dt_val, dIrem_dt_val, dUsc1_dt_val, dUsc2_dt_val]

# Arbitrary starting values
# Arbitrary starting values
Gpl_0 = 100
Ipl_0 = 10
Irem_0 = 5
Usc1_0 = 2
Usc2_0 = 1
y0 = np.array([Gpl_0, Ipl_0, Irem_0, Usc1_0, Usc2_0])

# Time span
t_span = (0, 120)

# Solve the system of equations
sol = solve_ivp(system_of_equations, t_span, y0)

# Access the solution
t = sol.t  # Array of time points
Gpl = sol.y[0]  # Array of Gpl values
Ipl = sol.y[1]  # Array of Ipl values
Irem = sol.y[2]  # Array of Irem values
Usc1 = sol.y[3]  # Array of Usc1 values
Usc2 = sol.y[4]  # Array of Usc2 values

# Plotting the graphs
plt.figure(figsize=(10, 6))
plt.plot(t, Gpl, label='Gpl')
plt.plot(t, Ipl, label='Ipl')
plt.plot(t, Irem, label='Irem')
plt.plot(t, Usc1, label='Usc1')

ValueError: diff requires input that is at least one dimensional